In [82]:
import sys
sys.path.append("../") # 상위 폴더의 model.py를 import 하기 위해 path 추가
from model import make_model
import pandas as pd
from kobert.utils import get_tokenizer
from gluonnlp.data import SentencepieceTokenizer

path='../data/ChatBotData.csv'

df = pd.read_csv(path, header=0, encoding='utf-8')
_, vocab = make_model(1)
sp = SentencepieceTokenizer(get_tokenizer())


using cached model
using cached model
using cached model


In [15]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [9]:
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [10]:
q_ids, a_ids = [], []
for i in range(len(df)):
    question = df.loc[i, 'Q']
    answer = df.loc[i, 'A']

    q_ids.append([vocab.token_to_idx[tok] for tok in sp(question)])
    a_ids.append([vocab.token_to_idx[tok] for tok in sp(answer)])

In [79]:
vocab.idx_to_token[0]

'[UNK]'

In [86]:
from collections import Counter

q_count = Counter()
for i in range(len(q_ids)):
    q_count += Counter(q_ids[i])

a_count = Counter()
for i in range(len(a_ids)):
    a_count += Counter(a_ids[i])

In [87]:
q_sorted = sorted(q_count.items(), key=lambda x: x[1], reverse=True)
a_sorted = sorted(a_count.items(), key=lambda x: x[1], reverse=True)

In [88]:
print('Among questions')
total_count=0
for i,(idx,count) in enumerate(q_sorted):
    if i < 10: 
        print('top {} frequent word: {} : {}'.format(i,vocab.idx_to_token[idx],count))
    if idx==vocab.token_to_idx['UNK']: 
        print('UNK count: {}'.format(count))
    total_count += count
print('total words count: {}, average length: {}'.format(total_count,total_count/len(q_ids)))

Among questions
top 0 frequent word: ▁ : 3852
top 1 frequent word: . : 2238
top 2 frequent word: 이 : 1817
top 3 frequent word: 가 : 1792
top 4 frequent word: ? : 1702
top 5 frequent word: 어 : 1358
top 6 frequent word: 까 : 1194
top 7 frequent word: 지 : 1140
top 8 frequent word: 네 : 876
top 9 frequent word: ▁좋아 : 847
UNK count: 249
total words count: 93131, average length: 7.877103949928106


In [89]:
print('Among answers')
total_count = 0
for i,(idx,count) in enumerate(a_sorted):
    if i < 10: 
        print('top {} frequent word: {}'.format(i,vocab.idx_to_token[idx]))
    if idx==vocab.token_to_idx['UNK']: 
        print('UNK count: {}'.format(count))
    total_count += count
print('total words count: {}, average length: {}'.format(total_count,total_count/len(a_ids)))

Among answers
top 0 frequent word: .
top 1 frequent word: 요
top 2 frequent word: 세요
top 3 frequent word: ▁
top 4 frequent word: 이
top 5 frequent word: 보
top 6 frequent word: 가
top 7 frequent word: ▁거
top 8 frequent word: 을
top 9 frequent word: 예
UNK count: 88
total words count: 109201, average length: 9.236319039160957


In [93]:
q_sentences, a_sentences = [], []
for i in range(len(df)):
    question = df.loc[i, 'Q']
    answer = df.loc[i, 'A']

    q_sentences.append(question)
    a_sentences.append(answer)

In [95]:
a_sentences[:5]

['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.']

In [91]:
# https://medium.com/@richardcsaky/neural-chatbots-are-dumb-65b6b40e9bd4
import math
from collections import Counter

In [97]:
def build_distribution(data):
  # Read data and store it in a list.
#   with open(data) as file:
#     clean_sentences = [' '.join(s.split()) for s in file]
  clean_sentences = data
  # Build a dict of unique sentences storing indexes to the original list.
  sentence_dict = {}
  for i, sentence in enumerate(clean_sentences):
    sentence_dict[sentence] = sentence_dict.get(sentence, [])
    sentence_dict[sentence].append(i)
    
  return sentence_dict, clean_sentences

source_distro, sources = build_distribution(q_sentences)
target_distro, targets = build_distribution(a_sentences)

In [124]:
print('source top 5 duplicated sentences')
print(sorted(source_distro.items(),key=lambda x: len(x[1]),reverse=True)[:5])
print('')
print('target top 5 duplicated sentences')
print(sorted(target_distro.items(),key=lambda x: len(x[1]),reverse=True)[:5])

source top 5 duplicated sentences
[('외로워', [3456, 3457, 7261, 7262]), ('너무 힘들다', [980, 5944, 5945]), ('올해 왜 이러지', [3387, 3388, 3389]), ('사랑을 했다', [9897, 9898, 9899]), ('결혼이나 하지 왜 자꾸 나한테 화 내냐구!', [152, 5527])]

target top 5 duplicated sentences
[('맛있게 드세요.', [45, 49, 50, 1263, 1267, 1270, 1271, 1390, 1391, 1888, 1889, 1972, 1976, 2245, 2246, 2678, 2680, 2682, 3371, 4436, 4437, 4581]), ('제가 있잖아요.', [626, 632, 824, 825, 1461, 1552, 3598, 3599, 4014, 4015, 4587, 4589, 4703, 4704, 6811, 8235, 8830]), ('조심하세요.', [409, 411, 989, 1699, 1700, 1761, 1762, 2171, 2172, 2243, 3820, 3924, 4891, 5062]), ('감기 조심하세요.', [1326, 1327, 1899, 1903, 2398, 2846, 3079, 3080, 3300, 4511, 4512, 4754, 5826, 6551]), ('잘할 수 있을 거예요.', [1190, 1191, 1948, 1949, 3822, 4980, 4981, 5221, 5228, 5230, 6050, 6512])]


In [109]:
def get_entropies(distro, pairs):
  entropies = {}

  for sentence, indices in distro.items():
    # Build a distribution for the current sentence, based on the targets.
    # 어떤 source sentence 가 나온 위치들에서의 target sentence들의 Counter를 구함.
    distribution = Counter([pairs[i] for i in indices])
    num_elements = len(indices)

    # Calculate entropy.
    entropy = 0
    for frequency in distribution.values():
      probability = frequency / num_elements
      entropy += probability * math.log(probability, 2)
    
    entropies[sentence] = -entropy
  return entropies

source_entropies = get_entropies(source_distro, targets)
target_entropies = get_entropies(target_distro, sources)

In [126]:
print('source 5 highest entropy sentences')
print(sorted(source_entropies.items(),key=lambda x: x[1],reverse=True)[:5])
print('')
print('target 5 highest entropy sentences')
print(sorted(target_entropies.items(),key=lambda x: x[1],reverse=True)[:5])

source 5 highest entropy sentences
[('올해 왜 이러지', 1.584962500721156), ('사랑을 했다', 1.584962500721156), ('외로워', 1.5), ('고양이 키우고 싶어', 1.0), ('공시 준비 힘들어', 1.0)]

target 5 highest entropy sentences
[('맛있게 드세요.', 4.459431618637295), ('제가 있잖아요.', 4.08746284125034), ('조심하세요.', 3.8073549220576055), ('감기 조심하세요.', 3.8073549220576055), ('잘할 수 있을 거예요.', 3.584962500721156)]


In [146]:
threshold = 1.0 # 2번 나온것들을 사용하지 않음. (0.5 * log_2(0.5))*2
source_refine=[]
target_refine=[]
for source, target in zip(sources, targets):
  # Here we can put whatever combination of the entropy thresholds.
  if source_entropies[source] < threshold and target_entropies[target] < threshold:
    source_refine.append(source)
    target_refine.append(target)

In [160]:
from pandas import DataFrame
# dummy label is added for the same format with original csv
refined_data = {'Q': source_refine, 'A': target_refine, 'label': [0]*len(source_refine)}
data_df = DataFrame(refined_data)

In [161]:
data_df.head

<bound method NDFrame.head of                             Q                                            A  \
0                         겁난다                                     용기 내보세요.   
1               공책 필기 나만 힘들어?                             성향 차이가 좀 있기는 하죠.   
2                기다리라고 말 못하겠어                              상대방의 선택에 맡겨보세요.   
3                   기분이 그지 같아                                신나는 음악 들어보세요.   
4                     기분이 더러워                                경쾌한 음악 들어보세요.   
...                       ...                                          ...   
4757   회사에서 어떤 사람이랑 자꾸 눈 마추쳐.                      눈 마주치는 게 우연인지 잘 살펴 보세요.   
4758      회식 중이라고 하는데 연락이 안돼.  정신 없이 바쁠지도 몰라요. 조금만 더 기다려보고 물어보는게 좋을 것 같아요.   
4759            후회 없이 사랑하고 싶어                                진심으로 다가가 보세요.   
4760              흑기사 해주는 짝남.                                       설렜겠어요.   
4761  힘든 연애 좋은 연애라는게 무슨 차이일까?                     잘 헤어질 수 있는 사이 여부인 거 같아요.   

      label  
0         0  
1    

In [162]:
data_df.to_csv('../data/ChatBotData_refined.csv', index=False)